# XRPL Fund Tokenization - Demo Notebook

This notebook walks through tokenizing a private fund on the XRP Ledger, step by step.

**Contents:**
1. Connect to XRPL Testnet
2. Create Cold (Issuer) and Hot (Operational) wallets
3. Configure the issuer account with proper flags
4. Create and authorize trust lines
5. Issue tokens
6. Simulate investor onboarding
7. Check balances and explore compliance features

---

## Setup: Imports and Configuration

We use the async versions of xrpl-py functions because Jupyter runs its own event loop. Actual scripts would use the sync versions

In [40]:
# Core imports - async versions for Jupyter compatibility
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.asyncio.wallet import generate_faucet_wallet
from xrpl.asyncio.transaction import submit_and_wait

# Transaction types
from xrpl.models.transactions import AccountSet, TrustSet, Payment
from xrpl.models.transactions.account_set import AccountSetAsfFlag
from xrpl.models.transactions.trust_set import TrustSetFlag

# Request types (for querying the ledger)
from xrpl.models.requests import AccountInfo, AccountLines, GatewayBalances

# Helpers
from xrpl.models.amounts import IssuedCurrencyAmount
from xrpl.utils import drops_to_xrp, xrp_to_drops

# Configuration
TESTNET_URL = "https://s.altnet.rippletest.net:51234"
CURRENCY_CODE = "IND"  # Our token's ticker symbol

print("✓ Imports loaded")
print(f"  Token symbol: {CURRENCY_CODE}")
print(f"  Network: Testnet")

✓ Imports loaded
  Token symbol: IND
  Network: Testnet


## Step 1: Connect to XRPL Testnet

Use the async JSON-RPC client to connect to Ripple's public testnet server.

In [41]:
# Create the client connection
client = AsyncJsonRpcClient(TESTNET_URL)

print(f"✓ Client created for {TESTNET_URL}")
print("  (Connection is established when we make our first request)")

✓ Client created for https://s.altnet.rippletest.net:51234
  (Connection is established when we make our first request)


## Step 2: Create and Fund Wallets

We need two wallets:
- **Cold Wallet (Issuer)**: The official issuer of fund tokens. In production, keys would be in cold storage.
- **Hot Wallet (Operational)**: Day-to-day operations, distributing tokens to investors.

The testnet faucet automatically funds each wallet with test XRP.

In [42]:
# Create Cold Wallet (Issuer)
print("Creating Cold Wallet (Issuer)...")
cold_wallet = await generate_faucet_wallet(client, debug=True)
print(f"\n✓ Cold Wallet created: {cold_wallet.address}")

Creating Cold Wallet (Issuer)...
Attempting to fund address rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
Faucet fund successful.

✓ Cold Wallet created: rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj


In [43]:
# Create Hot Wallet (Operational)
print("Creating Hot Wallet (Operational)...")
hot_wallet = await generate_faucet_wallet(client, debug=True)
print(f"\n✓ Hot Wallet created: {hot_wallet.address}")

Creating Hot Wallet (Operational)...
Attempting to fund address r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH
Faucet fund successful.

✓ Hot Wallet created: r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH


In [44]:
# Summary of our wallets
print("=" * 60)
print("WALLET SUMMARY")
print("=" * 60)
print(f"\nCold (Issuer):     {cold_wallet.address}")
print(f"Hot (Operational): {hot_wallet.address}")
print(f"\nView on explorer:")
print(f"  Cold: https://testnet.xrpl.org/accounts/{cold_wallet.address}")
print(f"  Hot:  https://testnet.xrpl.org/accounts/{hot_wallet.address}")

WALLET SUMMARY

Cold (Issuer):     rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
Hot (Operational): r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH

View on explorer:
  Cold: https://testnet.xrpl.org/accounts/rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
  Hot:  https://testnet.xrpl.org/accounts/r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH


## Step 3: Configure the Issuer (Cold Wallet)

Before issuing tokens, we need to set account flags on the cold wallet:

| Flag | Purpose |
|------|--------|
| `ASF_DEFAULT_RIPPLE` | **Required.** Allows tokens to transfer between third parties (not just to/from issuer) |
| `ASF_REQUIRE_AUTH` | **Required.** You must approve each trust line before it can hold tokens (KYC gate) |
| `ASF_ALLOW_TRUSTLINE_CLAWBACK` | Optional. Lets you recover tokens from any holder (regulatory compliance) |

In [45]:
# Enable Default Ripple - REQUIRED for token transfers between third parties
print("Enabling Default Ripple on issuer...")

default_ripple_tx = AccountSet(
    account=cold_wallet.address,
    set_flag=AccountSetAsfFlag.ASF_DEFAULT_RIPPLE,
)

response = await submit_and_wait(default_ripple_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Default Ripple enabled")
else:
    print(f"✗ Failed: {result}")

Enabling Default Ripple on issuer...

Result: tesSUCCESS
✓ Default Ripple enabled


In [46]:
# Enable Require Auth - Creates whitelist/KYC gate
print("Enabling Require Auth on issuer...")

require_auth_tx = AccountSet(
    account=cold_wallet.address,
    set_flag=AccountSetAsfFlag.ASF_REQUIRE_AUTH,
)

response = await submit_and_wait(require_auth_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Require Auth enabled")
    print("  (You must now authorize each trust line before tokens can flow)")
else:
    print(f"✗ Failed: {result}")

Enabling Require Auth on issuer...

Result: tesSUCCESS
✓ Require Auth enabled
  (You must now authorize each trust line before tokens can flow)


### Optional: Enable Clawback

Uncomment and run this cell to enable the ability to recover tokens from holders. **Must be done before any trust lines exist.**

In [47]:
# # OPTIONAL: Enable Clawback (must be done before any trust lines)
# print("Enabling Clawback on issuer...")

# clawback_tx = AccountSet(
#     account=cold_wallet.address,
#     set_flag=AccountSetAsfFlag.ASF_ALLOW_TRUSTLINE_CLAWBACK,
# )

# response = await submit_and_wait(clawback_tx, client, cold_wallet)
# result = response.result["meta"]["TransactionResult"]

# print(f"\nResult: {result}")
# if result == "tesSUCCESS":
#     print("✓ Clawback enabled")
#     print("  (You can now recover tokens from any holder if needed)")

## Step 4: Create Trust Line (Hot → Cold)

Before the hot wallet can receive tokens, it must create a **trust line** to the cold wallet.

Think of it as: *"I (hot wallet) trust the cold wallet's FUND tokens up to X amount."*

In [48]:
# Hot wallet creates trust line to cold wallet
print(f"Creating trust line: Hot → Cold for {CURRENCY_CODE}...")

TRUST_LIMIT = "1000000"  # Max tokens hot wallet can hold

trust_line_tx = TrustSet(
    account=hot_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=cold_wallet.address,
        value=TRUST_LIMIT,
    ),
)

response = await submit_and_wait(trust_line_tx, client, hot_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Trust line created")
    print(f"  Hot wallet can now hold up to {TRUST_LIMIT} {CURRENCY_CODE}")
    print(f"  (But tokens can't flow yet - issuer must authorize first)")
else:
    print(f"✗ Failed: {result}")

Creating trust line: Hot → Cold for IND...

Result: tesSUCCESS
✓ Trust line created
  Hot wallet can now hold up to 1000000000 IND
  (But tokens can't flow yet - issuer must authorize first)


## Step 5: Authorize the Trust Line

Since we enabled `Require Auth`, the cold wallet must explicitly approve the trust line before any tokens can flow.

This is your **KYC gate** — in production, you'd only authorize after verifying the holder.

In [49]:
# Cold wallet authorizes hot wallet's trust line
print("Authorizing trust line from issuer side...")

# The tfSetfAuth flag (0x00010000) tells XRPL to authorize this trust line
TF_SET_AUTH = 0x00010000

authorize_tx = TrustSet(
    account=cold_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=hot_wallet.address,  # Note: issuer field is the OTHER account
        value="0",  # Issuer doesn't need to set a limit
    ),
    flags=TF_SET_AUTH,
)

response = await submit_and_wait(authorize_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Trust line authorized")
    print("  Hot wallet can now receive FUND tokens from issuer")
else:
    print(f"✗ Failed: {result}")

Authorizing trust line from issuer side...

Result: tesSUCCESS
✓ Trust line authorized
  Hot wallet can now receive FUND tokens from issuer


## Step 6: Issue Tokens

Now the magic happens. When the cold wallet sends a payment to the hot wallet, **tokens are created**.

- Cold wallet's balance goes negative (it's an obligation)
- Hot wallet's balance goes positive (it's an asset)

In [50]:
# Issue tokens by sending payment from cold to hot
ISSUE_AMOUNT = "1000000"  # 1 million tokens

print(f"Issuing {ISSUE_AMOUNT} {CURRENCY_CODE} tokens...")

issue_tx = Payment(
    account=cold_wallet.address,
    destination=hot_wallet.address,
    amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=cold_wallet.address,
        value=ISSUE_AMOUNT,
    ),
)

response = await submit_and_wait(issue_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print(f"✓ Tokens issued!")
    print(f"  {ISSUE_AMOUNT} {CURRENCY_CODE} now exist")
    print(f"  Cold wallet obligation: -{ISSUE_AMOUNT}")
    print(f"  Hot wallet balance: +{ISSUE_AMOUNT}")
else:
    print(f"✗ Failed: {result}")

Issuing 1000000 IND tokens...

Result: tesSUCCESS
✓ Tokens issued!
  1000000 IND now exist
  Cold wallet obligation: -1000000
  Hot wallet balance: +1000000


## Step 7: Check Balances

Let's verify everything worked by checking balances from different perspectives.

In [51]:
# Check hot wallet's token balance (holder's perspective)
print("Checking Hot Wallet balances...\n")

response = await client.request(AccountLines(
    account=hot_wallet.address,
    ledger_index="validated",
))

print(f"Trust lines for {hot_wallet.address}:")
print("-" * 50)

for line in response.result.get("lines", []):
    currency = line["currency"]
    balance = line["balance"]
    issuer = line["account"]
    limit = line["limit"]
    
    print(f"  Currency: {currency}")
    print(f"  Balance:  {balance}")
    print(f"  Issuer:   {issuer}")
    print(f"  Limit:    {limit}")
    print()

Checking Hot Wallet balances...

Trust lines for r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH:
--------------------------------------------------
  Currency: IND
  Balance:  1000000
  Issuer:   rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
  Limit:    1000000000



In [52]:
# Check issuer's obligations (issuer's perspective)
print("Checking Cold Wallet (Issuer) obligations...\n")

response = await client.request(GatewayBalances(
    account=cold_wallet.address,
    ledger_index="validated",
))

obligations = response.result.get("obligations", {})
print(f"Total obligations (tokens issued):")
print("-" * 50)
for currency, amount in obligations.items():
    print(f"  {currency}: {amount}")

Checking Cold Wallet (Issuer) obligations...

Total obligations (tokens issued):
--------------------------------------------------
  IND: 1000000


---

## Simulate Investor Onboarding

Now let's simulate a new investor joining the fund:

1. Create investor wallet
2. Investor creates trust line to the token
3. You authorize their trust line (KYC approval)
4. Transfer tokens from hot wallet to investor

In [53]:
# Create an investor wallet
print("Creating Investor wallet...")
investor_wallet = await generate_faucet_wallet(client, debug=True)
print(f"\n✓ Investor wallet: {investor_wallet.address}")
print(f"  Explorer: https://testnet.xrpl.org/accounts/{investor_wallet.address}")

Creating Investor wallet...
Attempting to fund address rnj6YkT92mcbvFXzKrkWfeJScfEt1ZJHFd
Faucet fund successful.

✓ Investor wallet: rnj6YkT92mcbvFXzKrkWfeJScfEt1ZJHFd
  Explorer: https://testnet.xrpl.org/accounts/rnj6YkT92mcbvFXzKrkWfeJScfEt1ZJHFd


In [54]:
# Investor creates trust line to the fund token
print(f"Investor creating trust line for {CURRENCY_CODE}...")

investor_trust_tx = TrustSet(
    account=investor_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=cold_wallet.address,
        value="100000",  # Investor willing to hold up to 100k tokens
    ),
)

response = await submit_and_wait(investor_trust_tx, client, investor_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Investor trust line created")
    print("  (Waiting for issuer authorization...)")

Investor creating trust line for IND...

Result: tesSUCCESS
✓ Investor trust line created
  (Waiting for issuer authorization...)


In [55]:
# Fund manager authorizes investor (this is your KYC approval step)
print("Authorizing investor trust line (KYC approval)...")

TF_SET_AUTH = 0x00010000

authorize_investor_tx = TrustSet(
    account=cold_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=investor_wallet.address,
        value="0",
    ),
    flags=TF_SET_AUTH,
)

response = await submit_and_wait(authorize_investor_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Investor authorized")
    print("  Investor can now receive FUND tokens")

Authorizing investor trust line (KYC approval)...

Result: tesSUCCESS
✓ Investor authorized
  Investor can now receive FUND tokens


In [56]:
# Transfer tokens from hot wallet to investor
INVESTOR_ALLOCATION = "50000"  # 50k tokens

print(f"Transferring {INVESTOR_ALLOCATION} {CURRENCY_CODE} to investor...")

transfer_tx = Payment(
    account=hot_wallet.address,
    destination=investor_wallet.address,
    amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=cold_wallet.address,
        value=INVESTOR_ALLOCATION,
    ),
)

response = await submit_and_wait(transfer_tx, client, hot_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print(f"✓ Investor received {INVESTOR_ALLOCATION} {CURRENCY_CODE}")

Transferring 50000 IND to investor...

Result: tesSUCCESS
✓ Investor received 50000 IND


In [57]:
# Verify investor's balance
print("Checking investor's token balance...\n")

response = await client.request(AccountLines(
    account=investor_wallet.address,
    ledger_index="validated",
))

for line in response.result.get("lines", []):
    if line["currency"] == CURRENCY_CODE:
        print(f"✓ Investor holds: {line['balance']} {CURRENCY_CODE}")

Checking investor's token balance...

✓ Investor holds: 50000 IND


---

## Bonus: Compliance Features

### Freeze an Investor

If you need to halt trading for a specific investor (investigation, sanctions, etc.):

In [58]:
# Freeze investor's trust line
print("Freezing investor's trust line...")

freeze_tx = TrustSet(
    account=cold_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=investor_wallet.address,
        value="0",
    ),
    flags=TrustSetFlag.TF_SET_FREEZE,
)

response = await submit_and_wait(freeze_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Investor FROZEN")
    print("  They cannot send or receive FUND tokens until unfrozen")

Freezing investor's trust line...

Result: tesSUCCESS
✓ Investor FROZEN
  They cannot send or receive FUND tokens until unfrozen


In [59]:
# Try to send tokens to frozen investor (should fail)
print("Attempting to send tokens to frozen investor...")

test_transfer = Payment(
    account=hot_wallet.address,
    destination=investor_wallet.address,
    amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=cold_wallet.address,
        value="100",
    ),
)

response = await submit_and_wait(test_transfer, client, hot_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result != "tesSUCCESS":
    print("✓ Transfer blocked as expected (investor is frozen)")

Attempting to send tokens to frozen investor...

Result: tesSUCCESS


In [60]:
# Unfreeze investor
print("Unfreezing investor...")

unfreeze_tx = TrustSet(
    account=cold_wallet.address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=investor_wallet.address,
        value="0",
    ),
    flags=TrustSetFlag.TF_CLEAR_FREEZE,
)

response = await submit_and_wait(unfreeze_tx, client, cold_wallet)
result = response.result["meta"]["TransactionResult"]

print(f"\nResult: {result}")
if result == "tesSUCCESS":
    print("✓ Investor UNFROZEN")
    print("  They can now send and receive FUND tokens again")

Unfreezing investor...

Result: tesSUCCESS
✓ Investor UNFROZEN
  They can now send and receive FUND tokens again


---

## Summary

In [61]:
print("=" * 60)
print("FUND TOKENIZATION SUMMARY")
print("=" * 60)

print(f"\nToken: {CURRENCY_CODE}")
print(f"Network: XRPL Testnet")

print(f"\nWallets:")
print(f"  Cold (Issuer):     {cold_wallet.address}")
print(f"  Hot (Operational): {hot_wallet.address}")
print(f"  Investor:          {investor_wallet.address}")

print(f"\nExplorer Links:")
print(f"  Cold:     https://testnet.xrpl.org/accounts/{cold_wallet.address}")
print(f"  Hot:      https://testnet.xrpl.org/accounts/{hot_wallet.address}")
print(f"  Investor: https://testnet.xrpl.org/accounts/{investor_wallet.address}")

FUND TOKENIZATION SUMMARY

Token: IND
Network: XRPL Testnet

Wallets:
  Cold (Issuer):     rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
  Hot (Operational): r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH
  Investor:          rnj6YkT92mcbvFXzKrkWfeJScfEt1ZJHFd

Explorer Links:
  Cold:     https://testnet.xrpl.org/accounts/rP6fa1RvKCyT8usca3PqbpRkD4vbyBVySj
  Hot:      https://testnet.xrpl.org/accounts/r9ep7gwntqHoBudQZGAeBjNKU4NsMfGgpH
  Investor: https://testnet.xrpl.org/accounts/rnj6YkT92mcbvFXzKrkWfeJScfEt1ZJHFd


---

## Next Steps

Things to explore:

1. **Create more investors** - Run the investor onboarding cells again with new wallets
2. **Transfer between investors** - Try sending tokens from one investor to another
3. **Check the explorer** - Click the explorer links to see your accounts visually
4. **Try clawback** - Enable clawback (must recreate wallets first) and recover tokens
5. **Set transfer fees** - Use `AccountSet` with `transfer_rate` to charge fees on transfers